In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/skin_detection_1271')

Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
data = pd.read_csv('processed_data/ISIC_2019_Training_GroundTruth_Processed_Balanced.csv')

In [5]:
data

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,Cat,y
0,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
1,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL,1
2,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
3,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL,1
4,ISIC_0000007,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
...,...,...,...,...,...,...,...,...,...,...,...,...
24801,ISIC_0073153_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24802,ISIC_0073157_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24803,ISIC_0073198_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24804,ISIC_0073214_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1


In [6]:
def get_image(df):
    return np.asarray(Image.open((f"processed_data/{df['Cat']}/{df['image']}.jpg")), dtype=np.float32)

In [7]:
n_samples=1000; sample = data.sample(n_samples, random_state=42)
# sample = data #load the full dataset

X = sample.apply(get_image, axis=1)
X = np.stack(X)
y = sample.y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(700, 400, 400, 3) (300, 400, 400, 3) (700,) (300,)


Transfered model -Resnet

In [9]:
# Load the pre-trained ResNet50 model without the top (fully connected) layers
def load_model():
    model = ResNet50(weights='imagenet', include_top=False, input_shape=X_train[0].shape)
    return model

In [10]:
def set_nontrainable_layers(model):
    # Set the first layers to be untrainable
    model.trainable = False

    return model

In [11]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''

    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(200, activation='relu')
    prediction_layer = layers.Dense(1, activation='sigmoid')

    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])

    return model

In [12]:
def build_model():

    model = load_model()
    model = add_last_layers(model)

    opt = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy', 'Recall'])
    return model

In [13]:
model = build_model()
model.summary()

94765736/94765736 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 13, 13, 2048)      23587712  
                                                                 
 flatten (Flatten)           (None, 346112)            0         
                                                                 
 dense (Dense)               (None, 200)               69222600  
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 92,810,513
Trainable params: 69,222,801
Non-trainable params: 23,587,712
_________________________________________________________________


In [14]:
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

In [15]:
model = build_model()

es = EarlyStopping(monitor = 'val_accuracy',
                   mode = 'max',
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)


In [16]:
history = model.fit(X_train, y_train,
                    validation_split=0.3,
                    epochs=10,
                    batch_size=32,
                    callbacks=[es])

Epoch 1/10
16/16 [==============================] - 30s 971ms/step - loss: 22.2293 - accuracy: 0.6769 - recall: 0.7463 - val_loss: 7.9958 - val_accuracy: 0.7299 - val_recall: 0.8779
Epoch 2/10
16/16 [==============================] - 7s 437ms/step - loss: 2.1891 - accuracy: 0.8773 - recall: 0.9027 - val_loss: 9.3044 - val_accuracy: 0.7299 - val_recall: 0.9924
Epoch 3/10
16/16 [==============================] - 7s 428ms/step - loss: 1.2276 - accuracy: 0.9162 - recall: 0.9528 - val_loss: 5.0757 - val_accuracy: 0.6777 - val_recall: 0.6489
Epoch 4/10
16/16 [==============================] - 8s 511ms/step - loss: 0.1851 - accuracy: 0.9693 - recall: 0.9882 - val_loss: 5.2321 - val_accuracy: 0.7536 - val_recall: 0.8168
Epoch 5/10
16/16 [==============================] - 8s 485ms/step - loss: 0.0749 - accuracy: 0.9918 - recall: 0.9882 - val_loss: 5.7940 - val_accuracy: 0.7393 - val_recall: 0.8855
Epoch 6/10
16/16 [==============================] - 7s 465ms/step - loss: 0.0113 - accuracy: 0.995

### Evaluate on test set ###

In [17]:
print(model.evaluate(X_test, y_test, verbose=1))

10/10 [==============================] - 5s 473ms/step - loss: 3.4392 - accuracy: 0.7967 - recall: 0.8131
[3.4391520023345947, 0.79666668176651, 0.8130841255187988]
